#### Property of determinant

Use the properties of determinant

$$\det AB = \det A \det B$$

and

$$\det A = \prod \lambda_i(A)$$

we can compute determinant of a square matrix using Cholesky and LU with partial pivoting

#### Use Cholesky

For matrix $A$ that is `positive definite`, we have

$$A=LL^T$$

and

$$\det A = \det L \det L^T = \left(\prod L_{ii}\right)^2$$

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [ ]:
def cholesky_factorization(A):
    m = A.shape[0]
    l_mat = A.copy().astype(float)

    for k in range(m):
        if l_mat[k, k] <= 0:
            raise ValueError('Matrix is not positive definite')

        # Follow the first step, iteratively apply to a smaller and smaller K
        l_mat[k+1:, k+1:] -= np.outer(l_mat[k+1:, k], l_mat[k+1:, k]) / l_mat[k, k]
        l_mat[k:, k] /= np.sqrt(l_mat[k, k])

    return np.tril(l_mat)

In [ ]:
np.random.seed(42)
m = 40

A = np.random.randn(m, m)
A_sym = A.T @ A

try:
    l = cholesky_factorization(A_sym)
    det_A_sym = np.prod(np.diag(l))**2
    det_A_sym_np = np.linalg.det(A_sym)

    print(f'Determinant (Cholesky): {det_A_sym}')
    print(f'Determinant (NumPy): {det_A_sym_np}')

except Exception as e:
    print(e)

Determinant (Cholesky): 5.493123938057964e+44
Determinant (NumPy): 5.493123938093299e+44


#### Use LU with partial pivoting

For a `full rank square` matrix $A$, we find $P$, $L$, $U$ such that $PA=LU$

and, we have

$$\det PA = \det P \det A = \pm \det A$$

and

$$\det LU = \det L \det U = \prod u_{ii}$$

In [ ]:
def lu_factorization(A):
    m, n = A.shape
    u_mat = A.copy().astype(float)
    l_mat = np.identity(m)
    p_mat = np.identity(m)

    for k in range(m-1):
        # Find pivot
        pivot = np.argmax(np.abs(u_mat[k:, k])) + k

        if pivot != k:
            # Swap rows in u, p, and l
            u_mat[[k, pivot], :] = u_mat[[pivot, k], :]
            p_mat[[k, pivot], :] = p_mat[[pivot, k], :]
            l_mat[[k, pivot], :k] = l_mat[[pivot, k], :k]

        for j in range(k + 1, m):
            l_mat[j, k] = u_mat[j, k] / u_mat[k, k]
            # Subtract multiply of kth row from jth row
            u_mat[j, k:] -= l_mat[j, k] * u_mat[k, k:]

    return p_mat, l_mat, u_mat

In [ ]:
p, l, u = lu_factorization(A)

det_A = np.prod(np.diag(u))
det_A_np = np.linalg.det(A)

print(f'Determinant (LU): {det_A}')
print(f'Determinant (NumPy): {det_A_np}')

Determinant (LU): 2.3437414401089563e+22
Determinant (NumPy): -2.343741440108957e+22


##### Evaluate on a rank deficient matrix

In [ ]:
A[:,0] = A[:,-1]
p, l, u = lu_factorization(A)

det_A = np.prod(np.diag(u))
det_A_np = np.linalg.det(A)

print(f'Determinant (LU): {det_A}')
print(f'Determinant (NumPy): {det_A_np}')

Determinant (LU): -0.0
Determinant (NumPy): 12527193.411707794
